In [42]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from nomad.datamodel.datamodel import (
        EntryArchive,
    )
    from structlog.stdlib import (
        BoundLogger,
    )

import numpy as np
from nomad.config import config
from nomad.datamodel.data import ArchiveSection, EntryData, EntryDataCategory, Schema
from nomad.datamodel.metainfo.annotations import (
    ELNAnnotation,
    ELNComponentEnum,
    SectionProperties,
)
from nomad.datamodel.metainfo.basesections import (
    CompositeSystem,
    CompositeSystemReference,
    Process,
    System,
)
from nomad.metainfo import Datetime, MEnum, Quantity, SchemaPackage, Section, SubSection
from nomad.metainfo.metainfo import Category
from structlog.stdlib import BoundLogger
from nomad.datamodel.datamodel import (
        EntryArchive,
    )
from nomad.datamodel import EntryMetadata

In [43]:
from nomad_ikz_fz.schema_packages.mypackage import Feed_rod, FzCrystal, Fz_Materials, FzMaterial, Resistivity
import pandas as pd


In [44]:
df_fr_100=pd.read_excel('Inventarliste Rohstäbe (2).xlsx', sheet_name='100mm')
df_fr_100.fillna('', inplace=True)

In [45]:
df_fr_100#.columns

,Seriennummer 100mm,Länge cm,Status,Angespitzt,Geätzt,Ready to use,Bemerkung,Ort
0,44324001,,ready to use,+,+,+,,Wagen FZ-Halle
1,44318002,,muss geaetzt werden,+,-,-,test,Wagen FZ-Halle
2,ohne Nr.,,muss geaetzt werden,+,-,-,,Wagen FZ-Halle
3,533264003,,muss geaetzt werden,+,-,-,,Wagen FZ-Halle
4,533264005,,muss geaetzt werden,+,-,-,,Wagen FZ-Halle
5,ohne Nr.,,muss geaetzt werden,+,-,-,,Wagen FZ-Halle
6,ohne Nr.,,muss geaetzt werden,+,-,-,,Wagen FZ-Halle
7,ohne Nr.,77.0,muss geaetzt werden,+,-,-,,Keller
8,ohne Nr.,,muss geaetzt werden,+,-,-,,Keller
9,ohne Nr.,190.0,muss geaetzt werden,+,-,-,,Keller


In [5]:
import json

def process_dataframe(df):
    counter = 1

    for index, row in df.iterrows():
        fr = Feed_rod()  # Replace with the correct initialization of your object
        counter += 1
        if row['Seriennummer 100mm'] == 'ohne Nr.':
            fr.name = str(row['Seriennummer 100mm']) + str(counter)
        else:
            fr.name = str(row['Seriennummer 100mm'])

        if row['Länge cm '] != '':
            fr.length = float(row['Länge cm '])

        if row['Angespitzt'] == '+':
            fr.sharpened = True
        else:
            fr.sharpened = False
        if row['Geätzt'] == '+':
            fr.etched = True
        else:
            fr.etched = False

        if row["Bemerkung"] != '':
            fr.description = row["Bemerkung"]

        if row["Ort"] != '':
            fr.storage_location = row["Ort"]
        fr.diameter = "100 mm"

        archive = EntryArchive(metadata=EntryMetadata())
        fr.normalize(archive, None)

        with open("/home/brueckner/ikz-fz-nomad-plugin/feeds/" + fr.name + '.archive.json', 'w') as f:
            json.dump({'data': fr.m_to_dict(with_root_def=True)}, f, indent=2)

    print("Processing complete and files saved.")

# Example usage:
process_dataframe(df_fr_100)


Processing complete and files saved.


In [46]:
df_fr_126=pd.read_excel('Inventarliste Rohstäbe (2).xlsx', sheet_name='126-130mm')
df_fr_126.fillna('', inplace=True)
df_fr_126b

NameError: name 'df_fr_126b' is not defined

In [7]:
df_fr_126.Resistivity.unique()

array(['', 'Std Res', 'High Res ', 'High Res'], dtype=object)

In [47]:
df_fr_126.columns

Index(['Seriennummer 126-130mm', 'Resistivity', 'Länge cm ', 'Status',
       'Angespitzt', 'Geätzt', 'Ready to use', 'Bemerkung', 'Ort'],
      dtype='object')

In [6]:
import json

def process_dataframe_126(df):
    counter = 1

    for index, row in df.iterrows():
        fr = Feed_rod()  # Replace with the correct initialization of your object
        counter += 1

        if row['Seriennummer 126-130mm'] == 'ohne Nr.':
            fr.name = str(row['Seriennummer 126-130mm']) + str(counter)
        else:
            fr.name = str(row['Seriennummer 126-130mm'])

        if row['Länge cm '] != '':
            fr.length = float(row['Länge cm '])

        if row['Angespitzt'] == '+':
            fr.sharpened = True
        else:
            fr.sharpened = False

        if row['Geätzt'] == '+':
            fr.etched = True
        else:
            fr.etched = False

        if row["Bemerkung"] != '':
            fr.description = row["Bemerkung"]

        if row["Ort"] != '':
            fr.storage_location = row["Ort"]

        if row["Resistivity"] != '' and row["Resistivity"] == 'Std Res':
            fr.feed_rod_resistivity="standard resistivity"
        elif row["Resistivity"] != '' and row["Resistivity"] == 'High Res' or row["Resistivity"] == 'High Res ':
            fr.feed_rod_resistivity="high resistivity"
        fr.diameter = "126 - 130 mm"
        archive = EntryArchive(metadata=EntryMetadata())
        fr.normalize(archive, None)

        with open("/home/brueckner/ikz-fz-nomad-plugin/feeds/" + fr.name + '.archive.json', 'w') as f:
            json.dump({'data': fr.m_to_dict(with_root_def=True)}, f, indent=2)

    print("Processing complete and files saved.")

# Example usage:
process_dataframe_126(df_fr_126)


NameError: name 'df_fr_126' is not defined

In [48]:
df_crys= pd.read_excel('Kristalle_Inventar (3).xlsx')
df_crys.fillna('', inplace=True)
#df_crys
# Initialize empty lists to store resistivity, res_min, and res_max
resistivity = []
res_min = []
res_max = []

for index, row in df_crys.iterrows():
    value = row['Spez. Widerstand ρ [kΩcm]']
    try:
        # Try to convert the value to float
        float_value = float(value)
        resistivity.append(float_value)
        res_min.append(None)  # No min value for single float entries
        res_max.append(None)  # No max value for single float entries
    except ValueError:
        # If conversion fails, it's a string or not a float-convertible value
        if '-' in value:
            #print(value)
            parts = value.split('-')
            try:
                # Convert both parts to float and determine min and max
                part1 = float(parts[0].strip().replace(',', '.'))
                part2 = float(parts[1].strip().replace(',', '.'))
                min_value = min(part1, part2)
                #print(min_value)
                max_value = max(part1, part2)
                #print(max_value)
                res_min.append(min_value)
                res_max.append(max_value)
                resistivity.append(None)  # No single resistivity value for ranges
            except ValueError:
                # If conversion fails, the parts are not valid floats
                resistivity.append(None)
                res_min.append(None)
                res_max.append(None)
        else:
            resistivity.append(None)
            res_min.append(None)
            res_max.append(None)

# At this point, resistivity, res_min, and res_max lists contain the processed values
# You can add them to the DataFrame or use them as needed
df_crys['resistivity'] = resistivity
df_crys['res_min'] = res_min
df_crys['res_max'] = res_max
df_crys.fillna('', inplace=True)
df_crys

,Versuchsnummer,Bemerkung,Status,Anlage,Ist- Ø [mm],Length [mm],Orientierung,Spez. Widerstand ρ [kΩcm],L-Typ,Location,Datum,resistivity,res_min,res_max
0,REC II,,,,100.0,860.0,<100>,,,Kristallregal Züchtungshalle,NaT,,,
1,FZ 30#052,ein Teil verkauft :Matek,,FZ 30,105.0,560.0,<111> 3/4°,"36,6- 27,6",,Kristallregal Züchtungshalle,2021-12-13,,27.6,36.6
2,FZ 30#058,,,FZ 30,128.0,950.0,<111> KOH,0.005,,Kristallregal Züchtungshalle,2022-02-22,0.005,,
3,FZ 30#106,,,FZ 30,155.0,900.0,<100>,,,Kristallregal Züchtungshalle,2023-01-27,,,
4,FZ 30#100,,,FZ 30,155.0,950.0,<100>,,,Kristallregal Züchtungshalle,2023-01-16,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,VCGS7,,,,,,,,,Schrank Büro R. 124,NaT,,,
217,V2931,Fe + Mo dotiert,,FZ 1505-2,36.5,,<100>,,,Schrank Büro R. 124,2005-06-09,,,
218,V2932,Fe dotiert,,FZ 1505-2,37.0,,<100>,,,Schrank Büro R. 124,2005-06-10,,,
219,V2935,Mo dotiert,,FZ 1505-2,37.5,,<100>,,,Schrank Büro R. 124,2005-06-23,,,


In [4]:
df_crys.columns

Index(['Versuchsnummer', 'Bemerkung', 'Status', 'Anlage', 'Ist- Ø [mm] ',
       'Length [mm] ', 'Orientierung', 'Spez. Widerstand ρ [kΩcm]', 'L-Typ',
       'Location', 'Datum'],
      dtype='object')

In [60]:
df_crys#["L-Typ"].unique()

,Versuchsnummer,Bemerkung,Status,Anlage,Ist- Ø [mm],Length [mm],Orientierung,Spez. Widerstand ρ [kΩcm],L-Typ,Location,Datum,resistivity,res_min,res_max
0,REC II,,,,100.0,860.0,<100>,,,Kristallregal Züchtungshalle,NaT,,,
1,FZ 30#052,ein Teil verkauft :Matek,,FZ 30,105.0,560.0,<111> 3/4°,"36,6- 27,6",,Kristallregal Züchtungshalle,2021-12-13,,27.6,36.6
2,FZ 30#058,,,FZ 30,128.0,950.0,<111> KOH,0.005,,Kristallregal Züchtungshalle,2022-02-22,0.005,,
3,FZ 30#106,,,FZ 30,155.0,900.0,<100>,,,Kristallregal Züchtungshalle,2023-01-27,,,
4,FZ 30#100,,,FZ 30,155.0,950.0,<100>,,,Kristallregal Züchtungshalle,2023-01-16,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,VCGS7,,,,,,,,,Schrank Büro R. 124,NaT,,,
217,V2931,Fe + Mo dotiert,,FZ 1505-2,36.5,,<100>,,,Schrank Büro R. 124,2005-06-09,,,
218,V2932,Fe dotiert,,FZ 1505-2,37.0,,<100>,,,Schrank Büro R. 124,2005-06-10,,,
219,V2935,Mo dotiert,,FZ 1505-2,37.5,,<100>,,,Schrank Büro R. 124,2005-06-23,,,


In [61]:
import json

def process_dataframe_crys(df):
    counter = 1

    for index, row in df.iterrows():
        fr = FzCrystal()  # Replace with the correct initialization of your object
        counter += 1

        if str(row['Versuchsnummer']) != '':
            fr.name = str(row['Versuchsnummer'])

        if str(row['Anlage']) != '':
            fr.fz_furnace = str(row['Anlage'])
            print(fr.fz_furnace)


        if row["Orientierung"] != '':
            fr.orientation = str(row["Orientierung"])

        if row["L-Typ"] != '' and row["L-Typ"] == 'n' or row["L-Typ"] == 'p' or row["L-Typ"] == 'p/n':
            fr.doping_type = str(row["L-Typ"])
            # undoped add here
        else: fr.doping_type =  'undoped'

        if type(row["Datum"]) != pd._libs.tslibs.nattype.NaTType:
            fr.process_date = str(row["Datum"])

        fr.resistivity_measurement=Resistivity()

        if row["resistivity"] != '':
            fr.resistivity_measurement.resistivity = float(row["resistivity"])
        if row["res_min"] != '':
            fr.resistivity_measurement.resistivity_minimum = float(row["res_min"])
        if row["res_max"] != '':
            fr.resistivity_measurement.resistivity_maximum = float(row["res_max"])
        # if row["Spez. Widerstand ρ [kΩcm]"] != '':
        #     if '-' in row["Spez. Widerstand ρ [kΩcm]"]:
        #         fr.resistivity=row["Spez. Widerstand ρ [kΩcm]"].split('-')[0].replace(',', '.')
        #     else: fr.resistivity=row["Spez. Widerstand ρ [kΩcm]"].replace(',', '.')
        # #     fr.orientresistivityation = float(row["Spez. Widerstand ρ [kΩcm]"])

        if row['Length [mm] '] != '':
            fr.length = float(row['Length [mm] '])

        if row['Ist- Ø [mm] '] != '':
            fr.diameter = float(row['Ist- Ø [mm] '])

        if row["Bemerkung"] != '':
            fr.description = str(row["Bemerkung"])

        if row["Location"] != '':
            fr.location = row["Location"]

        archive = EntryArchive(metadata=EntryMetadata())
        fr.normalize(archive, None)

        with open('/home/brueckner/ikz-fz-nomad-plugin/crys/' +fr.name + '.archive.json', 'w') as f:
            json.dump({'data': fr.m_to_dict(with_root_def=True)}, f, indent=2)

    print("Processing complete and files saved.")

# Example usage:
process_dataframe_crys(df_crys)


FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1505-2
FZ 1520
?
FZ 1520
FZ 30
FZ 1505-2
FZ 1520
FZ 1520
FZ 1520
FZ 1505-2
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1505-2
FZ ZFA 80 II
FZ ZFA 80 II
FZ 1505-2
FZ 1520
FZ 1520
FZ 1505-2
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 1520
FZ 1520
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 30
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-1
FZ 1520
FZ 1505-1
FZ 1505-2
FZ 1505-2
FZ 1505-1
FZ 1505-2
FZ 1505-2
FZ 1520
FZ 1505-2
FZ ZFA 80 II
FZ 1505-1
FZ 1510
FZ 1505-1
FZ 1505-2
FZ 1505-1
FZ 1505-1
FZ 1505-2
FZ 1520
FZ 1520
FZ 1520
FZ 1520
FZ 1505-2
FZ 1505-1
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-2
FZ 1505-1
FZ 1505-2
FZ 1505-2
FZ 15

In [ ]:
/home/brueckner/ikz-fz-nomad-plugin/feeds/44318002.archive.json
crys

In [25]:
type(df_crys.Datum[0])==pd._libs.tslibs.nattype.NaTType

True

In [54]:
fr = FzCrystal()

In [55]:
archive = EntryArchive(metadata=EntryMetadata())
#fr.normalize(archive, None)

In [57]:
fr.resistivity_measurement=Resistivity()

In [58]:
fr.resistivity_measurement.resistivity=1.0

In [20]:
import pandas as pd

# Assuming df_crys is your DataFrame

# Initialize empty lists to store resistivity, res_min, and res_max
resistivity = []
res_min = []
res_max = []

for index, row in df_crys.iterrows():
    value = row['Spez. Widerstand ρ [kΩcm]']
    try:
        # Try to convert the value to float
        float_value = float(value)
        resistivity.append(float_value)
        res_min.append(None)  # No min value for single float entries
        res_max.append(None)  # No max value for single float entries
    except ValueError:
        # If conversion fails, it's a string or not a float-convertible value
        if '-' in value:
            print(value)
            parts = value.split('-')
            try:
                # Convert both parts to float and determine min and max
                part1 = float(parts[0].strip().replace(',', '.'))
                part2 = float(parts[1].strip().replace(',', '.'))
                min_value = min(part1, part2)
                print(min_value)
                max_value = max(part1, part2)
                print(max_value)
                res_min.append(min_value)
                res_max.append(max_value)
                resistivity.append(None)  # No single resistivity value for ranges
            except ValueError:
                # If conversion fails, the parts are not valid floats
                resistivity.append(None)
                res_min.append(None)
                res_max.append(None)
        else:
            resistivity.append(None)
            res_min.append(None)
            res_max.append(None)

# At this point, resistivity, res_min, and res_max lists contain the processed values
# You can add them to the DataFrame or use them as needed
df_crys['resistivity'] = resistivity
df_crys['res_min'] = res_min
df_crys['res_max'] = res_max

36,6- 27,6
27.6
36.6
1,5-1,8  
1.5
1.8
0,0002-0,0004
0.0002
0.0004
